<a href="https://colab.research.google.com/github/Foluwa/sql_assessment/blob/main/sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.9 MB/s eta 0:00:00


# TOPIC:  Hospital Management Database

In [2]:
import random
import datetime
import sqlite3
import numpy as np
import pandas as pd
from faker import Faker

In [3]:
# Initialize faker library
fake = Faker()

# Declare records
num_records = 1000
procedure_records = 20

# Medical fields for specialization
medical_fields = [
    'Anesthesiology', 'Cardiology', 'Dermatology', 'Endocrinology', 'Gastroenterology',
    'Hematology', 'Neurology', 'Oncology', 'Orthopedics', 'Pediatrics', 'Psychiatry',
    'Radiology', 'Surgery', 'Urology'
]

# PatientFeedbackScore
patient_feedback = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']

# List of dignosis types
diagnosis = ['Cold', 'Flu', 'Fever', 'Headache', 'Stomachache', 'Gastritis']

# Procedure names
procedures = ['X-ray', 'Blood Test', 'Physical Therapy', 'Appendectomy', 'Cesarean Section', 'Colonoscopy',
              'Endoscopy', 'Laparoscopy', 'Angiography', 'Cardiac Catheterization', 'Echocardiogram',
              'Electrocardiogram (ECG or EKG)', 'Biopsy', 'Dialysis', 'Blood Transfusion', 'Bone Marrow Transplant', 'Colonoscopy',  'Ultrasound', 'Mammogram', 'Pap Smear']
procedure_names = [random.choice(procedures) for _ in range(20)]

# Gender list
gender = ['Male', 'Female']

# Ethnicity List
ethnicity = ['Caucasian', 'African American', 'Asian', 'Hispanic', 'Other']

# Patient severity level list
severity_level = ['Mild', 'Moderate', 'Severe']

# Doctors IDs
DoctorIDs = range(1, num_records + 1)

# Patients IDs
PatientIDs = [random.randint(10000000, 99999999) for _ in range(num_records)]

# Procedure IDs
procedure_ids = [random.randint(100, 1000) for _ in range(20)]

# Procedure cost
procedure_cost = [random.randint(100, 10000) for _ in range(20)]


In [4]:
def random_date(start_date, end_date):
    """ Define function to generate random date within a range """
    return start_date + datetime.timedelta(
        seconds=random.randint(0, int((end_date - start_date).total_seconds())))

def format_phone_number(phone_number):
    """ Function to format phone number with country code """
    # Add country code if missing
    if not phone_number.startswith('+'):
        phone_number = '+1' + phone_number[1:]
    return phone_number

def generate_patient_data(num_records):
    """ Function to generate patient data """
    patient_data = {
        'PatientID': PatientIDs,
        'PatientName': [fake.name() for _ in range(num_records)],
        'Gender': [fake.random_element(gender) for _ in range(num_records)],
        'DateOfBirth': [fake.date_of_birth(minimum_age=18, maximum_age=90) for _ in range(num_records)],
        'PhoneNumber': [format_phone_number(fake.phone_number()) for _ in range(num_records)],
        'Ethnicity': [fake.random_element(ethnicity) for _ in range(num_records)],
        'AdmissionDate': [random_date(datetime.datetime(2020, 1, 1), datetime.datetime(2023, 1, 1)) for _ in range(num_records)],
        'InsuranceID': [fake.random_number(digits=10) for _ in range(num_records)],
        'SeverityLevel': [random.choice(severity_level) for _ in range(num_records)],
        'HospitalStayDuration': np.random.randint(1, 30, size=num_records),
        'TotalCost': np.round(np.random.uniform(1000, 10000, size=num_records), 2)
    }
    return patient_data

# Generate patient data
patients_df = pd.DataFrame(generate_patient_data(num_records))

# Set PatientID as primary key
patients_df.set_index('PatientID', inplace=True)

# Display the DataFrame
patients_df.head()

,PatientName,Gender,DateOfBirth,PhoneNumber,Ethnicity,AdmissionDate,InsuranceID,SeverityLevel,HospitalStayDuration,TotalCost
PatientID,,,,,,,,,,
15802889,Christopher Flynn,Female,1955-12-11,+101-500-383-6418x270,Caucasian,2022-05-31 12:17:42,2235512051,Moderate,12,8887.83
54280936,Kathy Cordova,Male,1983-01-22,+1914698848,Asian,2020-12-30 11:25:32,444504094,Severe,24,3664.90
96177254,Brandi Reid,Female,1962-07-09,+101-448-735-0306x101,Caucasian,2020-06-09 15:22:21,3336388259,Mild,19,7628.11
11024476,Walter Morales,Female,1990-01-18,+160.716.8491,Hispanic,2020-08-04 17:21:41,3766380656,Moderate,11,2293.10
23329507,James Daniel,Male,1935-07-14,+143.285.0971x490,African American,2020-08-23 04:14:41,7259360923,Severe,23,5187.79


In [5]:
def generate_doctor_data(num_records):
    """ Function to generate data for Doctors Table """
    data = {
        'DoctorID': DoctorIDs,
        'DoctorName': [fake.name() for _ in range(num_records)],
        'Specialization': [random.choice(medical_fields) for _ in range(num_records)],
        'ExperienceYears': np.random.randint(1, 30, size=num_records),
        'AverageRating': np.round(np.random.uniform(1, 5, size=num_records), 2),
        'PatientFeedbackScore': [random.choice(patient_feedback) for _ in range(num_records)],
        'AverageConsultationDuration': np.random.randint(10, 60, size=num_records)
    }
    return data

def create_doctors_dataframe(num_records):
    """ Function to create Doctors DataFrame """
    data = generate_doctor_data(num_records)
    doctors_df = pd.DataFrame(data)
    return doctors_df

# Generate Doctors DataFrame with 100 records
doctors_df = create_doctors_dataframe(num_records)

# Set DoctorID as primary key
doctors_df.set_index('DoctorID', inplace=True)

# Display the DataFrame
doctors_df.head()

,DoctorName,Specialization,ExperienceYears,AverageRating,PatientFeedbackScore,AverageConsultationDuration
DoctorID,,,,,,
1,Christina Griffin,Pediatrics,15,1.63,Poor,42
2,Susan Spence,Endocrinology,16,4.84,Poor,41
3,Jonathan Hughes,Anesthesiology,1,3.22,Poor,25
4,Robert Martin,Orthopedics,29,3.17,Good,28
5,Lawrence Garcia,Cardiology,3,4.37,Excellent,31


In [6]:
def gen_appointments_data():
    """ Generate data for Appointments Table """
    appointments_data = {
        'AppointmentID': range(1, num_records+1),
        'PatientID': [random.choice(PatientIDs) for _ in range(num_records)],
        'DoctorID': [random.choice(DoctorIDs) for _ in range(num_records)],
        'AppointmentDate': [datetime.datetime(2024, np.random.randint(1, 13), np.random.randint(1, 29)) for _ in range(num_records)],
        'ConsultationFee': np.round(np.random.uniform(50, 200, size=num_records), 2),
        'Diagnosis': [random.choice(diagnosis) for _ in range(num_records)],
        'TreatmentDuration': np.random.randint(10, 60, size=num_records)
    }
    return appointments_data

appointments_data = gen_appointments_data()

# Create DataFrame for Appointments Table
appointments_df = pd.DataFrame(appointments_data)

# Create a compound key by combining PatientID and AppointmentDate
appointments_df['AppointmentDate'] = pd.to_datetime(appointments_df['AppointmentDate']).dt.date
appointments_df['CompoundKey'] = appointments_df['PatientID'].astype(str) + '-' + appointments_df['AppointmentDate'].astype(str)

# Set PatientID and DoctorID as foreign keys
appointments_df.set_index(['PatientID', 'AppointmentDate'], inplace=True)
appointments_df['DoctorID'] = appointments_df['DoctorID'].astype('int64')
appointments_df = appointments_df.join(doctors_df, on='DoctorID')

# Display the DataFrame
appointments_df.head()

,,AppointmentID,DoctorID,ConsultationFee,Diagnosis,TreatmentDuration,CompoundKey,DoctorName,Specialization,ExperienceYears,AverageRating,PatientFeedbackScore,AverageConsultationDuration
PatientID,AppointmentDate,,,,,,,,,,,,
36956559,2024-01-28,1,1000,146.94,Headache,15,36956559-2024-01-28,Nicholas Jones,Psychiatry,2,3.68,Fair,23
76167814,2024-07-13,2,433,183.90,Cold,27,76167814-2024-07-13,Jaime Jimenez,Orthopedics,21,4.90,Good,15
95173409,2024-12-22,3,684,98.67,Cold,19,95173409-2024-12-22,Roy Lewis,Neurology,2,4.16,Good,10
12987634,2024-07-08,4,605,88.53,Gastritis,30,12987634-2024-07-08,Marie Curtis,Urology,7,1.52,Very Good,33
92295666,2024-02-21,5,373,152.03,Fever,42,92295666-2024-02-21,Sarah Contreras,Endocrinology,6,2.33,Very Good,18


In [7]:
def create_procedure_data():
    """ Procedures Table """
    procedures_data = {
        'ProcedureID': procedure_ids,
        'ProcedureName': procedure_names,
        'ProcedureCost': procedure_cost
    }
    return procedures_data

procedures_data = create_procedure_data()
procedures_df = pd.DataFrame(procedures_data)
procedures_df.set_index('ProcedureID', inplace=True)

# Display the DataFrame
procedures_df.head()

,ProcedureName,ProcedureCost
ProcedureID,,
306,Blood Transfusion,7423
607,Mammogram,1958
424,Appendectomy,6866
274,Appendectomy,2532
572,Blood Transfusion,2087


In [8]:
def generate_future_date():
    """ Function to generate future appointment dates """
    return datetime.datetime.now() + datetime.timedelta(days=random.randint(1, 30))

def generate_appointment_procedures_data():
    """ Generate data for AppointmentProcedures Table (Junction Table) """
    appointment_procedures_data = {
        'PatientID': [random.choice(PatientIDs) for _ in range(num_records)],
        'AppointmentDate': [generate_future_date().date() for _ in range(num_records)],
        'ProcedureID': [random.choice(procedure_ids) for _ in range(num_records)],
        'Quantity': [random.randint(1, 5) for _ in range(num_records)]
    }
    return appointment_procedures_data

appointment_procedures_data = generate_appointment_procedures_data()

# Create DataFrame for AppointmentProcedures Table
appointment_procedures_df = pd.DataFrame(appointment_procedures_data)

# Display the DataFrame
appointment_procedures_df.head()

,PatientID,AppointmentDate,ProcedureID,Quantity
0,90668935,2024-03-13,607,2
1,61649205,2024-03-29,593,4
2,16637174,2024-03-22,593,3
3,65017093,2024-03-10,607,3
4,22313660,2024-03-23,863,3


In [9]:
def export_to_sqlite(patients_df, doctors_df, appointments_df, procedures_df, appointment_procedures_df, db_file='hospital_data.db'):
    """ Function to export CSV to SQLite DB """
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Export Patients DataFrame to SQL
    patients_df.to_sql('Patients', conn, if_exists='replace', index=True,  )

    # Export Doctors DataFrame to SQL
    doctors_df.to_sql('Doctors', conn, if_exists='replace', index=True)

    # Export Appointments DataFrame to SQL
    appointments_df.to_sql('Appointments', conn, if_exists='replace', index=True)

    # Export Procedures DataFrame to SQL
    procedures_df.to_sql('Procedures', conn, if_exists='replace', index=True)

    # Export AppointmentProcedures DataFrame to SQL
    appointment_procedures_df.to_sql('AppointmentProcedures', conn, if_exists='replace', index=True)

    # Commit changes and close connection
    conn.commit()
    return conn.close()

# Export to SQL
export_to_sqlite(patients_df, doctors_df, appointments_df, procedures_df, appointment_procedures_df)
